### Setup

In [1]:
from os import getcwd

import tensorflow as tf
import tensorflow_datasets as tfds

### One-Hot encoding

In [4]:
def my_one_hot(feature, label):

  one_hot = tf.one_hot(label, depth=3) 
  return feature, one_hot

# Testing 
_, one_hot = my_one_hot(["a", "b", "c", "a"], [1, 2, 3, 1])
print(one_hot)

tf.Tensor(
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 1. 0.]], shape=(4, 3), dtype=float32)


### Loading dataset

In [5]:
train_data = tfds.load("rock_paper_scissors:3.*.*", split = "train", as_supervised=True)
val_data = tfds.load("rock_paper_scissors:3.*.*", split = "test", as_supervised=True)

# Testing

train_data_len = len(list(train_data))
val_data_len = len(list(val_data))

print(train_data_len)
print(val_data_len)

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteSBRT0C/rock_paper_scissors-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteSBRT0C/rock_paper_scissors-test.tfrecord


Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.
2520
372


### Mapping one hot encode function to dataset

In [6]:
train_data = train_data.map(lambda features, labels: my_one_hot(features, labels))
val_data = val_data.map(lambda features, labels: my_one_hot(features, labels))

print(type(train_data))

<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


### Exploring dataset metadata

In [9]:
_, info = tfds.load(name="rock_paper_scissors:3.*.*", split="train", as_supervised=True, with_info=True)

print(info.features['image'].shape)

(300, 300, 3)


### Training your simple CNN classifier

In [10]:
train_batches = train_data.shuffle(100).batch(10)
validation_batches = val_data.batch(32)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)), # YOUR CODE HERE), 
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    # YOUR CODE HERE - Add a maxpool2d layer with kernel (2,2) 
    # YOUR CODE HERE - Add a flatten layer                                           
    tf.keras.layers.Dense(3, activation='softmax')# YOUR CODE HERE)  # Remember there are 3 classes to classify and to use proper activation
])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
flatten (Flatten)            (None, 355216)            0         
_________________________________________________________________
dense (Dense)                (None, 3)                 1065651   
Total params: 1,066,099
Trainable params: 1,066,099
Non-trainable params: 0
_________________________________________________________________


In [11]:
EPOCH = 3

# You should get decent enough training accuracy in 3-4 epochs itself as this one layer model will heavily overfit

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])# YOUR CODE HERE)

history = model.fit(train_batches, epochs= EPOCH , validation_data=validation_batches, validation_steps=1)
    
print("Final Training Accuracy:-",history.history['accuracy'][-1])
print("Final Validation Accuracy:-",history.history['val_accuracy'][-1])

Epoch 1/3
252/252 [==============================] - 84s 332ms/step - loss: 2697.9837 - accuracy: 0.6474 - val_loss: 14.3030 - val_accuracy: 0.6875
Epoch 2/3
252/252 [==============================] - 83s 330ms/step - loss: 0.1276 - accuracy: 0.9884 - val_loss: 17.1459 - val_accuracy: 0.6250
Epoch 3/3
252/252 [==============================] - 83s 329ms/step - loss: 0.0328 - accuracy: 0.9968 - val_loss: 22.4733 - val_accuracy: 0.6250
Final Training Accuracy:- 0.9988095164299011
Final Validation Accuracy:- 0.625
